In [1]:
import pandas as pd

In [8]:
count = 0
def handle_surrogates(text):
    return text.encode('utf-16', 'surrogatepass').decode('utf-16')
with open('../private_data/downloaded_tweets.txt', 'r', encoding='utf-16') as f:
    response = eval(eval(f.readline()))
    tweets_df = pd.DataFrame(columns=['author_id', 'tweet_id', 'tweet_text'])
    users_df = pd.DataFrame(columns=['author_id', 'location'])
    for tweet in response['data']:
        # Handle surrogate pairs
        tweets_df = tweets_df.append({
            'author_id': tweet['author_id'],
            'tweet_id': tweet['id'],
            'tweet_text': handle_surrogates(tweet['text'])
        }, ignore_index=True)
    for user in response['includes']['users']:
        try:
            users_df = users_df.append({
                'author_id': user['id'],
                'location': handle_surrogates(user['location'])
            }, ignore_index=True)
        except KeyError: # user doesn't have location string
            continue
users_df.head()

,author_id,location
0,2743106511,"North Carolina, USA"
1,1018650672353312769,"Los Angeles, CA"
2,292404883,"Olmsted Falls, OH"
3,1209320835535822848,earth usa🐈🐾🌊
4,1026847067849871360,"Wisconsin, USA"


In [ ]:
with open('../private_data/downloaded_tweets.txt', 'r', encoding='utf-16') as f:
    tweet_list = [eval(eval(s)) for s in f.readlines()]
for d in tweet_list:
    for tweet in d['data']:
        try:
            print(tweet['text'])
        except UnicodeEncodeError:
            print(tweet['text'].encode('utf-16', 'surrogatepass').decode('utf-16'))